# Faster R-CNN (TensorFlow Object Detection API) — End-to-End Fine-tuning Notebook

Tệp nguồn này thiết lập một pipeline fine-tuning có thể tái lập cho Faster R-CNN + ResNet50 + FPN sử dụng TensorFlow Object Detection API (TF2). Chạy các cell theo thứ tự trên xuống

**Những việc sẽ làm trong chương trình**:

- Cài đặt môi trường và TensorFlow Object Detection API
- Tải một tập dữ liệu nhỏ (**PennFudanPed**) và chuyển đổi sang định dạng TFRecord, đây sẽ được dùng làm tập huấn luyện
- Tạo tệp **label_map.pbtxt** là tệp chứa nhãn các đối tượng trong tập dữ liệu huấn luyện mới
- Tải **checkpoint** đã huấn luyện sẵn (**Faster R-CNN ResNet50 V1 FPN**) từ **TF2 Model Zoo**
- Tự động tạo **file pipeline.config** tương thích
- Huấn luyện mô hình bằng **model_main_tf2.py**
- Xuất **SavedModel** bằng exporter_main_v2.py
- Chạy **inference** và trực quan hóa kết quả detection


> ⚠️ Recommended: Run on GPU (Colab or local with CUDA). Tested with **TF 2.13–2.15**.


## 1) Runtime check & project paths

In [ ]:

import os, sys

PROJECT_ROOT = os.path.abspath(".")
WORK_DIR = os.path.join(PROJECT_ROOT, "tf_frcnn_work")
DATA_DIR = os.path.join(WORK_DIR, "data")
MODEL_DIR = os.path.join(WORK_DIR, "model_dir")
EXPORT_DIR = os.path.join(WORK_DIR, "exported")
PIPELINE_DIR = os.path.join(WORK_DIR, "pipeline")
CKPT_DIR = os.path.join(WORK_DIR, "pretrained_ckpt")
OD_API_DIR = os.path.join(WORK_DIR, "models")  # if we clone the TF models repo

for d in [WORK_DIR, DATA_DIR, MODEL_DIR, EXPORT_DIR, PIPELINE_DIR, CKPT_DIR]:
    os.makedirs(d, exist_ok=True)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("WORK_DIR:", WORK_DIR)
print("Using Python:", sys.version)


## 2) Install TensorFlow & Object Detection API

We try two approaches:

- **Preferred (simple)**: pip install OD API subpackage.
- **Fallback**: clone repo & compile protos.


In [ ]:

import sys, subprocess, os, shutil

def run(cmd):
    print(">>>", cmd)
    r = subprocess.run(cmd, shell=True, check=False)
    print("Return code:", r.returncode)
    return r.returncode == 0

ok = run("pip install --upgrade pip")
ok = run("pip install 'tensorflow>=2.13,<2.16'")
ok = run("pip install Cython contextlib2 pillow lxml jupyter matplotlib pandas tf-models-official==2.15.0 --no-deps")
ok = run("pip install protobuf==4.25.3")
ok = run("pip install --no-cache-dir 'git+https://github.com/tensorflow/models.git#egg=object-detection&subdirectory=research/object_detection'")

if not ok:
    print("\nDirect pip install failed. Cloning repo & building protos as fallback...")
    os.makedirs(OD_API_DIR, exist_ok=True)
    os.chdir(WORK_DIR)
    run("git clone --depth 1 https://github.com/tensorflow/models.git")
    os.chdir(os.path.join(WORK_DIR, "models", "research"))
    run("apt-get update && apt-get install -y protobuf-compiler")  # may fail on non-Debian systems; ignore errors
    run("protoc object_detection/protos/*.proto --python_out=.")
    run("pip install .")
    os.chdir(PROJECT_ROOT)

# Simple import test
test_code = "import object_detection\nprint('Object Detection API import: OK')\n"
with open(os.path.join(WORK_DIR, "od_import_test.py"), "w") as f:
    f.write(test_code)
run(f"{sys.executable} {os.path.join(WORK_DIR, 'od_import_test.py')}")


## 3) Download dataset (PennFudanPed) & convert to TFRecord
#####
- Mục đích: Chuẩn bị dữ liệu đầu vào cho mô hình.
- Giải thích: TF Object Detection API sử dụng định dạng TFRecord — một định dạng nhị phân tối ưu để TensorFlow đọc nhanh trong quá trình huấn luyện. Bộ dữ liệu ví dụ PennFudanPed chứa ảnh người đi bộ và annotation bounding boxes.
    - Tải dataset gốc (thường ở dạng ảnh + file annotation VOC hoặc JSON).
    - Dùng script chuyển đổi annotation sang định dạng TFRecord kèm thông tin nhãn, tọa độ bounding box.

In [ ]:

import os, zipfile, random, shutil, urllib.request, pathlib, glob, json
from PIL import Image
random.seed(1337)

raw_dir = os.path.join(DATA_DIR, "raw")
img_dir = os.path.join(DATA_DIR, "images")
tfrecord_dir = os.path.join(DATA_DIR, "tfrecord")
os.makedirs(raw_dir, exist_ok=True)
os.makedirs(img_dir, exist_ok=True)
os.makedirs(tfrecord_dir, exist_ok=True)

# Download
url = "https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip"
zip_path = os.path.join(raw_dir, "PennFudanPed.zip")
if not os.path.exists(zip_path):
    print("Downloading:", url)
    urllib.request.urlretrieve(url, zip_path)
else:
    print("Already downloaded:", zip_path)

# Extract
extract_root = os.path.join(raw_dir, "PennFudanPed")
if not os.path.exists(extract_root):
    print("Extracting...")
    with zipfile.ZipFile(zip_path, 'r') as zf:
        zf.extractall(raw_dir)
else:
    print("Already extracted:", extract_root)

png_dir = os.path.join(extract_root, "PNGImages")
mask_dir = os.path.join(extract_root, "PedMasks")

def masks_to_boxes(mask_img):
    import numpy as np
    arr = np.array(mask_img)
    inst_ids = np.unique(arr[:,:,0])
    inst_ids = [i for i in inst_ids if i != 0]
    boxes = []
    for iid in inst_ids:
        ys, xs = (arr[:,:,0] == iid).nonzero()
        if len(xs)==0 or len(ys)==0:
            continue
        x1, y1, x2, y2 = xs.min(), ys.min(), xs.max(), ys.max()
        boxes.append((x1,y1,x2,y2))
    return boxes

meta = []
imgs = sorted(glob.glob(os.path.join(png_dir, '*.png')))
for p in imgs:
    name = os.path.splitext(os.path.basename(p))[0]
    mask_path = os.path.join(mask_dir, name + "_mask.png")
    if not os.path.exists(mask_path): 
        continue
    im = Image.open(p).convert("RGB")
    w,h = im.size
    jpg_path = os.path.join(img_dir, name + ".jpg")
    if not os.path.exists(jpg_path):
        im.save(jpg_path, quality=95)
    mask = Image.open(mask_path).convert("RGB")
    boxes = masks_to_boxes(mask)
    anns = [{"category":"person","bbox":[int(x1),int(y1),int(x2),int(y2)]} for (x1,y1,x2,y2) in boxes]
    meta.append({"file_name": os.path.basename(jpg_path), "width": w, "height": h, "annotations": anns})

with open(os.path.join(DATA_DIR, "pennfudan_meta.json"), "w") as f:
    json.dump(meta, f, indent=2)

random.shuffle(meta)
split = int(0.8*len(meta))
train_meta = meta[:split]
val_meta = meta[split:]
with open(os.path.join(DATA_DIR, "train.json"), "w") as f:
    json.dump(train_meta, f, indent=2)
with open(os.path.join(DATA_DIR, "val.json"), "w") as f:
    json.dump(val_meta, f, indent=2)

print("Train:", len(train_meta), "Val:", len(val_meta))

# label_map
label_map_path = os.path.join(DATA_DIR, "label_map.pbtxt")
with open(label_map_path, "w") as f:
    f.write('item { id: 1 name: "person" }\n')
print("Wrote", label_map_path)

# TFRecord writer
import tensorflow as tf

def create_tf_example(rec, label_map):
    filename = rec["file_name"].encode("utf8")
    width = rec["width"]; height = rec["height"]
    xmins, xmaxs, ymins, ymaxs, classes_text, classes = [], [], [], [], [], []
    for a in rec["annotations"]:
        x1,y1,x2,y2 = a["bbox"]
        xmins.append(x1/width); xmaxs.append(x2/width)
        ymins.append(y1/height); ymaxs.append(y2/height)
        classes_text.append(a["category"].encode("utf8"))
        classes.append(label_map[a["category"]])
    with tf.io.gfile.GFile(os.path.join(img_dir, rec["file_name"]), "rb") as fid:
        encoded_jpg = fid.read()

    def bytes_feature(v): return tf.train.Feature(bytes_list=tf.train.BytesList(value=[v]))
    def float_list_feature(v): return tf.train.Feature(float_list=tf.train.FloatList(value=v))
    def int64_list_feature(v): return tf.train.Feature(int64_list=tf.train.Int64List(value=v))

    example = tf.train.Example(features=tf.train.Features(feature={
        "image/height": tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        "image/width": tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        "image/filename": bytes_feature(filename),
        "image/source_id": bytes_feature(filename),
        "image/encoded": bytes_feature(encoded_jpg),
        "image/format": bytes_feature(b"jpg"),
        "image/object/bbox/xmin": float_list_feature(xmins),
        "image/object/bbox/xmax": float_list_feature(xmaxs),
        "image/object/bbox/ymin": float_list_feature(ymins),
        "image/object/bbox/ymax": float_list_feature(ymaxs),
        "image/object/class/text": tf.train.Feature(bytes_list=tf.train.BytesList(value=classes_text)),
        "image/object/class/label": int64_list_feature(classes),
    }))
    return example

label_map = {"person":1}
def write_tfrecord(meta_list, out_path):
    with tf.io.TFRecordWriter(out_path) as w:
        for r in meta_list:
            w.write(create_tf_example(r, label_map).SerializeToString())

train_record = os.path.join(tfrecord_dir, "train.record")
val_record   = os.path.join(tfrecord_dir, "val.record")
write_tfrecord(train_meta, train_record)
write_tfrecord(val_meta, val_record)
print("Wrote TFRecords:", train_record, val_record)


## 4) Download pretrained Faster R-CNN checkpoint (TF2 Model Zoo)
##### 
- Mục đích: Tận dụng mô hình đã được pretrain để rút ngắn thời gian huấn luyện và tăng độ chính xác.
- Giải thích: TF2 Model Zoo cung cấp checkpoint pretrained trên COCO hoặc ImageNet. Chọn mô hình Faster R-CNN ResNet50 V1 FPN để fine-tune trên dataset mới. Điều này giúp mô hình bắt đầu từ trọng số đã học được đặc trưng chung, chỉ tinh chỉnh thêm cho dataset của bạn.

In [ ]:

import os, urllib.request, tarfile

MODEL_ZOO_URL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz"
tar_path = os.path.join(CKPT_DIR, os.path.basename(MODEL_ZOO_URL))

if not os.path.exists(tar_path):
    print("Downloading:", MODEL_ZOO_URL)
    urllib.request.urlretrieve(MODEL_ZOO_URL, tar_path)
else:
    print("Already downloaded:", tar_path)

extract_path = os.path.join(CKPT_DIR, "faster_rcnn_resnet50_v1_640x640_coco17_tpu-8")
if not os.path.exists(extract_path):
    print("Extracting...")
    with tarfile.open(tar_path, "r:gz") as tfp:
        tfp.extractall(CKPT_DIR)
else:
    print("Already extracted:", extract_path)

print("Checkpoint dir:", extract_path)


## 5) Generate `pipeline.config`
#####
- Mục đích: Tạo file cấu hình cho quá trình huấn luyện.
- Giải thích: pipeline.config chứa tất cả thông số:
    - Model config (loại backbone, FPN, số class)
    - Train config (batch size, learning rate, optimizer, số step)
    - Train input config (đường dẫn TFRecord train, augmentation)
    - Eval input config (đường dẫn TFRecord eval) 
      API hỗ trợ script để sinh tự động dựa trên checkpoint đã tải.

In [ ]:

import os

NUM_CLASSES = 1
NUM_STEPS = 3000

label_map = os.path.join(DATA_DIR, "label_map.pbtxt")
train_record = os.path.join(DATA_DIR, "tfrecord", "train.record")
val_record   = os.path.join(DATA_DIR, "tfrecord", "val.record")
ckpt_base = os.path.join(CKPT_DIR, "faster_rcnn_resnet50_v1_640x640_coco17_tpu-8")
ckpt_path = os.path.join(ckpt_base, "checkpoint", "ckpt-0")

pipeline_template = r"""# Auto-generated pipeline.config for Faster R-CNN ResNet50 V1 FPN (TF2)
model {
  faster_rcnn {
    num_classes: __NUM_CLASSES__
    image_resizer { keep_aspect_ratio_resizer { min_dimension: 640 max_dimension: 640 } }
    feature_extractor { type: "faster_rcnn_resnet50_keras" }
    first_stage_anchor_generator {
      grid_anchor_generator { scales: [0.25, 0.5, 1.0, 2.0] aspect_ratios: [0.5, 1.0, 2.0] }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer { l2_regularizer { weight: 0.0004 } }
      initializer { truncated_normal_initializer { stddev: 0.01 } }
      activation: RELU_6
    }
    first_stage_nms_iou_threshold: 0.7
    first_stage_nms_score_threshold: 0.0
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 14
    maxpool_kernel_size: 2
    maxpool_stride: 2
    second_stage_box_predictor { mask_rcnn_box_predictor { fc_hyperparams { op: FC regularizer { l2_regularizer { weight: 0.0004 } } initializer { variance_scaling_initializer {} } activation: RELU } } }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.05
        iou_threshold: 0.5
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SIGMOID
    }
    second_stage_localization_loss_weight: 2.0
    second_stage_classification_loss_weight: 1.0
  }
}

train_config {
  batch_size: 2
  num_steps: __NUM_STEPS__
  optimizer {
    momentum_optimizer {
      learning_rate { cosine_decay_learning_rate { learning_rate_base: 0.02 total_steps: __NUM_STEPS__ warmup_learning_rate: 0.006 warmup_steps: 500 } }
      momentum: 0.9
    }
    use_moving_average: false
  }
  fine_tune_checkpoint: "__FINE_TUNE_CKPT__"
  fine_tune_checkpoint_type: "detection"
  data_augmentation_options { random_horizontal_flip {} }
  data_augmentation_options { random_adjust_brightness {} }
}

train_input_reader {
  label_map_path: "__LABEL_MAP__"
  tf_record_input_reader { input_path: "__TRAIN_RECORD__" }
}

eval_config {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
  max_evals: 1
}

eval_input_reader {
  label_map_path: "__LABEL_MAP__"
  shuffle: false
  num_readers: 1
  tf_record_input_reader { input_path: "__VAL_RECORD__" }
}"""

pipeline_text = (pipeline_template
    .replace("__NUM_CLASSES__", str(NUM_CLASSES))
    .replace("__NUM_STEPS__", str(NUM_STEPS))
    .replace("__FINE_TUNE_CKPT__", ckpt_path.replace("\\","/"))
    .replace("__LABEL_MAP__", label_map.replace("\\","/"))
    .replace("__TRAIN_RECORD__", train_record.replace("\\","/"))
    .replace("__VAL_RECORD__", val_record.replace("\\","/"))
)

os.makedirs(PIPELINE_DIR, exist_ok=True)
pipeline_path = os.path.join(PIPELINE_DIR, "pipeline.config")
with open(pipeline_path, "w") as f:
    f.write(pipeline_text)

print("Wrote pipeline:", pipeline_path)
print("Fine-tune checkpoint:", ckpt_path)
print("Train record:", train_record)
print("Val record:", val_record)
print("Label map:", label_map)


## 6) Train (model_main_tf2.py): 
##### File nguồn này đã có sẵn khi tải git https://github.com/tensorflow/models/blob/master/research/object_detection/model_main_tf2.py
- Mục đích: Huấn luyện mô hình trên dữ liệu của bạn.
- Giải thích: Script model_main_tf2.py sẽ:
    - Đọc pipeline.config
    - Nạp pretrained checkpoint
    - Chạy huấn luyện (training loop) và validation theo từng step
    - Lưu checkpoint mới theo chu kỳ để có thể resume hoặc xuất mô hình.

In [ ]:

import sys, os, subprocess, shutil

def which(script):
    return shutil.which(script)

entry_model_main = which("model_main_tf2.py")
entry_exporter   = which("exporter_main_v2.py")
if entry_model_main is None or entry_exporter is None:
    guess = os.path.join(WORK_DIR, "models", "research", "object_detection")
    alt_model_main = os.path.join(guess, "model_main_tf2.py")
    alt_exporter   = os.path.join(guess, "exporter_main_v2.py")
    if os.path.exists(alt_model_main): entry_model_main = alt_model_main
    if os.path.exists(alt_exporter):   entry_exporter   = alt_exporter

print("model_main_tf2.py path:", entry_model_main)
print("exporter_main_v2.py path:", entry_exporter)

assert entry_model_main and os.path.exists(entry_model_main), "Cannot find model_main_tf2.py"
assert entry_exporter and os.path.exists(entry_exporter), "Cannot find exporter_main_v2.py"

os.makedirs(MODEL_DIR, exist_ok=True)

cmd = f'"{sys.executable}" "{entry_model_main}" --model_dir="{MODEL_DIR}" --pipeline_config_path="{os.path.join(PIPELINE_DIR, "pipeline.config")}" --num_train_steps=3000 --sample_1_of_n_eval_examples=1'
print("Run this to start training:")
print(cmd)
# Uncomment the line below to actually run training inside the notebook cell
# subprocess.run(cmd, shell=True, check=True)


## 7) Export SavedModel (exporter_main_v2.py)
##### Tương tự, file nguồn này đã có sẵn khi tải git https://github.com/tensorflow/models/blob/master/research/object_detection/exporter_main_v2.py
- Mục đích: Xuất mô hình đã huấn luyện ra định dạng SavedModel để triển khai hoặc inference.
- Giải thích: exporter_main_v2.py lấy checkpoint cuối, gói toàn bộ graph + trọng số thành một thư mục SavedModel mà TensorFlow Serving, TFLite hay TF Hub có thể dùng.

In [ ]:

import os, sys, glob, subprocess

entry_exporter = shutil.which("exporter_main_v2.py")
if entry_exporter is None:
    guess = os.path.join(WORK_DIR, "models", "research", "object_detection", "exporter_main_v2.py")
    if os.path.exists(guess):
        entry_exporter = guess

if not os.path.exists(MODEL_DIR):
    raise SystemExit("MODEL_DIR not found. Train first.")

cmd = f'"{sys.executable}" "{entry_exporter}" --input_type=image_tensor --pipeline_config_path="{os.path.join(PIPELINE_DIR, "pipeline.config")}" --trained_checkpoint_dir="{MODEL_DIR}" --output_directory="{EXPORT_DIR}"'
print("Run this to export SavedModel:")
print(cmd)
# Uncomment to execute:
# subprocess.run(cmd, shell=True, check=True)


## 8) Inference & visualization on a few validation images
#####
- Mục đích: Kiểm tra kết quả, chạy thử mô hình đã fine-tune và xem bounding boxes trực quan.
- Giải thích: Dùng SavedModel đã xuất để dự đoán trên ảnh mới. Sau đó, dùng OpenCV hoặc Matplotlib vẽ khung (rectangle) và nhãn class (putText) lên ảnh để kiểm tra chất lượng mô hình.

In [ ]:

import os, glob, json
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import tensorflow as tf

# locate any saved_model under EXPORT_DIR
saved_model_dir = None
for p in glob.glob(os.path.join(EXPORT_DIR, "*")):
    sm = os.path.join(p, "saved_model")
    if os.path.isdir(sm):
        saved_model_dir = sm; break

if not saved_model_dir:
    print("No exported SavedModel found. Export first.")
else:
    print("Loading:", saved_model_dir)
    detect_fn = tf.saved_model.load(saved_model_dir)

    with open(os.path.join(DATA_DIR, "val.json")) as f:
        val_meta = json.load(f)

    os.makedirs("viz_results", exist_ok=True)
    for rec in val_meta[:10]:
        ip = os.path.join(DATA_DIR, "images", rec["file_name"])
        img = Image.open(ip).convert("RGB")
        np_img = np.array(img)
        out = detect_fn(tf.convert_to_tensor(np_img)[None, ...])

        boxes = out["detection_boxes"][0].numpy()
        scores = out["detection_scores"][0].numpy()
        classes = out["detection_classes"][0].numpy().astype(int)

        H,W = np_img.shape[:2]
        draw = img.copy(); D = ImageDraw.Draw(draw)
        for b,s,c in zip(boxes, scores, classes):
            if s < 0.5: continue
            y1,x1,y2,x2 = b
            x1,y1,x2,y2 = x1*W, y1*H, x2*W, y2*H
            D.rectangle([x1,y1,x2,y2], outline=(255,0,0), width=3)
            D.text((x1,y1), f"person:{s:.2f}", fill=(255,0,0))
        out_path = os.path.join("viz_results", f"viz_{rec['file_name']}")
        draw.save(out_path)
        print("Saved:", out_path)


## 9) Tips & Troubleshooting

- Nếu `protoc` bị thiếu, hãy cài đặt nó (Colab: `apt-get install -y protobuf-compiler`, Windows: tải xuống protoc và thêm vào PATH).
- Ghim các phiên bản tương thích nếu bạn thấy vấn đề phụ thuộc: `pip install "tensorflow>=2.13,<2.16" numpy<2.0`.
- Tăng `NUM_STEPS` và kích thước lô để có độ chính xác cao hơn (hãy chú ý đến bộ nhớ GPU).
- Mở rộng `label_map.pbtxt` và chuyển đổi TFRecord khi bạn có nhiều lớp hơn.
